In [ ]:
# !pip install -q datasets  # tokenizers is optional, we default to a tiny char tokenizer
#!tt-smi -r
import os, sys, math, random, textwrap
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import List, Tuple, Dict, Optional
from datasets import load_dataset

# Point PYTHONPATH to your TTML build (adjust if needed)
import ttnn
import ttml
from ttml.common.config import TrainingConfig, TransformerConfig, get_training_config, get_model_config

# Repro
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)

set_seed(42)
os.chdir(f"{os.environ['TT_METAL_HOME']}/tt-train")

In [ ]:
TRAINING_CONFIG="training_shakespeare_nanogpt.yaml"

In [ ]:
from datasets import load_dataset

def load_shakespeare_text() -> str:

    ds = load_dataset(
        "text",
        data_files={"train": f'{os.environ["TT_METAL_HOME"]}/tt-train/data/shakespeare.txt'},
    )
    text = "\n".join(ds["train"]["text"])
    return text
# --- Character-level tokenizer (default) ---
class CharTokenizer:
    def __init__(self, text: str):
        # Sorted stable alphabet for reproducibility
        chars = sorted(list(set(text)))
        self.stoi: Dict[str, int] = {ch: i for i, ch in enumerate(chars)}
        self.itos: List[str] = chars

    @property
    def vocab_size(self) -> int:
        return len(self.itos)

    def encode(self, s: str) -> List[int]:
        return [self.stoi[c] for c in s]

    def decode(self, ids: List[int]) -> str:
        return "".join(self.itos[i] for i in ids)

# --- Optional: BPE tokenizer using HF `tokenizers` ---
def build_bpe_tokenizer_from_text(tmp_corpus_path: str = "shakespeare_corpus.txt"):
    # Minimal training pipeline; produces subword tokens (not pure char)
    # Only used if cfg.use_bpe=True
    from tokenizers import Tokenizer
    from tokenizers.models import BPE
    from tokenizers.trainers import BpeTrainer
    from tokenizers.pre_tokenizers import ByteLevel

    tok = Tokenizer(BPE(unk_token="[UNK]"))
    tok.pre_tokenizer = ByteLevel()
    trainer = BpeTrainer(vocab_size=8192, special_tokens=["[UNK]", "[BOS]", "[EOS]"])
    tok.train(files=[tmp_corpus_path], trainer=trainer)
    return tok

# --- Build tokenizer + dataset splits ---
def prepare_data(cfg: TrainingConfig):
    text = load_shakespeare_text()

    if cfg.use_bpe:
        # Train a tiny BPE tokenizer on the fly
        with open("shakespeare_corpus.txt", "w", encoding="utf-8") as f:
            f.write(text)
        bpe = build_bpe_tokenizer_from_text("shakespeare_corpus.txt")

        def encode_all(t: str) -> np.ndarray:
            return np.array(bpe.encode(t).ids, dtype=np.uint32)

        def decode(ids: List[int]) -> str:
            return bpe.decode(ids)

        vocab_size = bpe.get_vocab_size()
        encode, decode_fn = encode_all, decode
    else:
        # Default: fast, deterministic character-level tokenizer
        ctok = CharTokenizer(text)
        vocab_size = (ctok.vocab_size + 31) // 32 * 32  # pad to multiple of 32
        encode = lambda t: np.array(ctok.encode(t), dtype=np.uint32)
        decode_fn = lambda ids: ctok.decode(list(ids))

    # Encode full corpus, split 90/10
    ids = encode(text)
    n = len(ids)
    n_train = int(n * 0.9)
    train_ids = ids[:n_train]
    val_ids = ids[n_train:]
    return train_ids, val_ids, vocab_size, decode_fn

train_config = get_training_config(TRAINING_CONFIG)
train_ids, val_ids, vocab_size, decode = prepare_data(train_config)
vocab_size, len(train_ids), len(val_ids)


In [ ]:
def get_batch(split_ids: np.ndarray, seq_len: int, batch_size: int) -> Tuple[np.ndarray, np.ndarray]:
    n = len(split_ids) - seq_len - 1
    ix = np.random.randint(0, n, size=(batch_size,))
    x = np.stack([split_ids[i:i+seq_len] for i in ix], axis=0)             # [B, T]
    y = np.stack([split_ids[i+1:i+seq_len+1] for i in ix], axis=0)         # [B, T] next-token targets
    return x.astype(np.uint32), y.astype(np.uint32)

def build_causal_mask(T: int) -> np.ndarray:
    # [1,1,T,T] float32 with 1s for allowed positions (i >= j), else 0
    m = np.tril(np.ones((T, T), dtype=np.float32))
    return m.reshape(1, 1, T, T)


In [ ]:
def create_model_and_optim(model_cfg: TransformerConfig, training_cfg: TrainingConfig, vocab_size: int):
    # GPT2 config via your bindings
    gcfg = ttml.models.gpt2.GPT2TransformerConfig()
    gcfg.num_heads = model_cfg.num_heads
    gcfg.embedding_dim = model_cfg.embedding_dim
    gcfg.num_blocks = model_cfg.num_blocks
    gcfg.vocab_size = int(vocab_size)
    gcfg.max_sequence_length = model_cfg.max_sequence_length
    gcfg.dropout_prob = 0.2
    # optional flags exist (runner_type, weight_tying, positional_embedding_type, experimental, ...)
    # we keep defaults for a minimal demo

    model = ttml.models.gpt2.create_gpt2_model(gcfg)

    # AdamW via your bindings
    adamw_cfg = ttml.optimizers.AdamWConfig.make(
        float(training_cfg.lr),
        float(training_cfg.beta1),
        float(training_cfg.beta2),
        float(training_cfg.eps),
        float(training_cfg.weight_decay),
    )
    optim = ttml.optimizers.AdamW(model.parameters(), adamw_cfg)
    return model, optim

transformer_config = get_model_config(train_config.model_config)
model, optim = create_model_and_optim(transformer_config, train_config, vocab_size)
model


In [ ]:
def train(cfg: TrainingConfig, model, optim, seq_len: int, train_ids: np.ndarray, val_ids: np.ndarray):
    set_seed(cfg.seed)
    loss_fn = ttml.ops.loss.cross_entropy_loss
    reduce = ttml.ops.ReduceType.MEAN

    causal_mask = build_causal_mask(seq_len)
    tt_mask = ttml.autograd.Tensor.from_numpy(causal_mask, ttnn.Layout.TILE, ttnn.DataType.BFLOAT16)   # [1,1,T,T], float32

    model.train()
    train_losses: List[float] = []
    val_losses: List[float] = []

    for step in range(1, cfg.steps + 1):
        # ---- batch ----
        x_u32, y_u32 = get_batch(train_ids, seq_len, cfg.batch_size)
        # TTML shapes: inputs [B,1,1,T] (uint32), targets [B,T] (int32)
        tt_x = ttml.autograd.Tensor.from_numpy(x_u32.reshape(cfg.batch_size, 1, 1, seq_len), ttnn.Layout.ROW_MAJOR, ttnn.DataType.UINT32)
        tt_y = ttml.autograd.Tensor.from_numpy(y_u32, ttnn.Layout.ROW_MAJOR, ttnn.DataType.UINT32)  # [B,T] uint32

        # ---- forward/backward ----
        optim.zero_grad()
        logits = model(tt_x, tt_mask)   
    
        loss = loss_fn(logits, tt_y, reduce)
        loss.backward(False)
        ttml.autograd.AutoContext.get_instance().reset_graph()
        optim.step()

        train_losses.append(float(loss.to_numpy()))

        # ---- occasional eval on val set ----
        
        if (step % cfg.eval_every) == 0 or step == 1:
            model.eval()
            with np.errstate(all='ignore'):
                vx_u32, vy_u32 = get_batch(val_ids, seq_len, cfg.batch_size)
                vtt_x = ttml.autograd.Tensor.from_numpy(vx_u32.reshape(cfg.batch_size, 1, 1, seq_len), ttnn.Layout.ROW_MAJOR, ttnn.DataType.UINT32)
                vtt_y = ttml.autograd.Tensor.from_numpy(vy_u32, ttnn.Layout.ROW_MAJOR, ttnn.DataType.UINT32)
                vlogits = model(vtt_x, tt_mask)
    
                vloss = loss_fn(vlogits, vtt_y, reduce)
                val_losses.append(float(vloss.to_numpy()))
            
            model.train()

            print(f"step {step:5d} | train_loss {train_losses[-1]:.4f} | val_loss {val_losses[-1]:.4f}")
        

    return train_losses, val_losses

train_losses, val_losses = train(train_config, model, optim, transformer_config.max_sequence_length, train_ids, val_ids)
len(train_losses), len(val_losses)


In [ ]:
# Loss curve
plt.figure()
plt.plot(np.arange(1, len(train_losses)+1), train_losses, label="train")
if len(val_losses) > 0:
    # approximate x-locations for val points
    x_val = np.linspace(1, len(train_losses), num=len(val_losses))
    plt.plot(x_val, val_losses, marker="o", linestyle="--", label="val")
plt.xlabel("Step")
plt.ylabel("Cross-Entropy Loss")
plt.title("NanoGPT on Shakespeare (char-level)")
plt.legend()
plt.tight_layout()
plt.show()

# Greedy sampling (for a quick sanity check)
def sample_greedy(model, decode_fn, start: str, max_new_tokens: int, seq_len: int):
    # simple char-level sampling with a running window
    model.eval()
    # build running context from start prompt (encode to uint32)
    ctx_ids = [ord(c) for c in start]  # placeholder; replaced next line when using our tokenizer
    # We don't know the tokenizer here; reconstruct via decode/encode roundtrip from dataset
    # A safe approach: pass through the training pipeline's tokenizer again.
    # Here, we simply assume `decode` is paired with the same encode used earlier,
    # so we'll re-encode using the train text mapping:
    # Since we don't have encode() now, we carry the last batch as context.
    # For a concise demo we'll just start from spaces if empty:
    if len(start) == 0:
        start = " "
    # Using last validation chunk as seed context for correctness
    # (In practice you'd keep the tokenizer encode() callable; omitted to keep demo minimal.)
    text_seed = start

    # Small helper: keep an internal list of token ids we feed
    # For simplicity, use the most recent validation chunk as initial tokens
    x_u32, _ = get_batch(val_ids, seq_len, 1)   # [1, T]
    running = x_u32[0].tolist()

    # Warm-start by replacing first few tokens with our prompt, if short
    prompt_ids = running[:]
    # No direct encode() handle here; we’ll just trust the random seed batch

    for _ in range(max_new_tokens):
        inp = np.array(running[-seq_len:], dtype=np.uint32).reshape(1,1,1,seq_len)
        tt_inp = ttml.autograd.Tensor.from_numpy(inp)
        causal_mask = build_causal_mask(seq_len)
        tt_mask = ttml.autograd.Tensor.from_numpy(causal_mask)

        logits = model(tt_inp, tt_mask)

        # logits -> numpy [1, seq_len, vocab] or similar; take last position
        np_logits = logits.to_numpy()
        last_logits = np_logits.reshape(1, -1, vocab_size)[:, -1, :]  # [1, V]
        next_id = int(np.argmax(last_logits, axis=-1)[0])
        running.append(next_id)
        ttml.autograd.AutoContext.get_instance().reset_graph()

    # Decode only the newly generated tail for display (best-effort)
    gen_ids = running[-max_new_tokens:]
    print("\nGenerated (greedy):\n")
    print(textwrap.fill(decode(gen_ids), width=100))

# Quick sample (greedy)
sample_greedy(model, decode, start="ROMEO:\n", max_new_tokens=300, seq_len=cfg.seq_len)
